In [6]:
import requests
import pandas as pd

def fetch_fred_data(start_date="2000-01-01", end_date="2025-01-01", api_key_path="fred_api_key.txt"):
    """Fetch economic indicators from the FRED API and convert all to monthly before merging."""

    # Define economic indicators with their frequencies
    series_ids = {
        "US Grocery Sales": ("RSGCS", "M"),  # Monthly
        "CPI (Food at Home)": ("CUSR0000SAF11", "M"),  # Monthly
        "GDP": ("GDPC1", "Q")  # Quarterly (Needs conversion)
    }

    # Read API key
    with open(api_key_path, "r") as file:
        API_KEY = file.read().strip()

    df_list = []

    for name, (series_id, frequency) in series_ids.items():
        url = (f"https://api.stlouisfed.org/fred/series/observations?"
               f"series_id={series_id}&api_key={API_KEY}&file_type=json&"
               f"observation_start={start_date}&observation_end={end_date}")
        
        response = requests.get(url)
        data = response.json()

        if "observations" not in data:
            print(f"❌ No data retrieved for {name}")
            continue

        # Convert API response to DataFrame
        df = pd.DataFrame(data["observations"])
        df["date"] = pd.to_datetime(df["date"])
        df[name] = pd.to_numeric(df["value"], errors="coerce")
        df = df[["date", name]]

        # ✅ Debug: Print first few rows before processing
        print(f"\n📌 Raw Data for {name} (before processing):")
        print(df.head())

        # ✅ Convert data to Monthly before merging
        df.set_index("date", inplace=True)

        if frequency == "Q":  # Convert quarterly to monthly using forward-fill
            df = df.resample("ME").ffill()

        # ✅ Debug: Print first few rows after conversion
        print(f"\n✅ Converted {name} to Monthly:")
        print(df.head())

        df_list.append(df)

    if not df_list:
        print("❌ No data retrieved for any series. Returning an empty DataFrame.")
        return pd.DataFrame(columns=["date"] + list(series_ids.keys()))  # Return empty DataFrame

    # ✅ Merge all datasets on 'date'
    final_df = df_list[0]
    for df in df_list[1:]:
        final_df = final_df.merge(df, on="date", how="outer")

    # ✅ Debug: Print merged DataFrame before returning
    print("\n📊 Final Merged Data Preview:")
    print(final_df.head())

    return final_df


NameError: name 'final_df' is not defined